# Installation

For installation instructions please refer to `Simply laced diagrams` notebook.

In [1]:
using Pkg

In [2]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 7840U w/ Radeon  780M Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver3)
Threads: 8 default, 0 interactive, 4 GC (on 16 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8


In [3]:
Pkg.activate(joinpath(@__DIR__, ".."))

  Activating project at `~/Mathematics/Research/Property (T)/Chevalley/2306.12358`


In [4]:
Pkg.status()

Status `~/Mathematics/Research/Property (T)/Chevalley/2306.12358/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [1e616198] COSMO v0.8.9
  [5d8bd718] Groups v0.8.0
  [7073ff75] IJulia v1.25.0
  [4076af6c] JuMP v1.22.2
  [03b72c93] PropertyT v0.6.0 `https://github.com/kalmarek/PropertyT.jl#master`
  [c946c3f1] SCS v2.0.0
⌃ [3f2553a9] SCS_MKL_jll v3.2.4+1 ⚲
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [5]:
using Groups
import Groups.MatrixGroups

In [6]:
using PropertyT
using PropertyT.PG # PermutationGroups
using PropertyT.IntervalArithmetic

> In this notebook only rudimentary commentary is included. For the extended one please consult `Simply laced diagrams` notebook.

# $G₂$ as matrix group

We will first define $G₂$ by explicit matrix generators.

The following [GAP](https://www.gap-system.org/) code has been used to obtain matrices of adjoint operators with respect to a basis of the $\mathfrak{g}_2$ algebra.

```GAP
gap> g2 := SimpleLieAlgebra("G", 2, Rationals);
<Lie algebra of dimension 14 over Rationals>
gap> g2rs := RootSystem(g2);
<root system of rank 2>
gap> positive := PositiveRootVectors(g2rs);
[ v.1, v.2, v.3, v.4, v.5, v.6 ]
gap> negative := NegativeRootVectors(g2rs);
[ v.7, v.8, v.9, v.10, v.11, v.12 ]
gap> all_gens := ShallowCopy(positive);; Append(all_gens, negative);; all_gens;
[ v.1, v.2, v.3, v.4, v.5, v.6, v.7, v.8, v.9, v.10, v.11, v.12 ]
gap> adj_mats := List(all_gens, x->AdjointMatrix(Basis(g2), x));;
```

With this data we can define the group $G_2$ as the matrix group generated by exponentials of `adj_mats`:

In [7]:
include(joinpath(@__DIR__, "..", "src", "G₂_gens.jl"));

In [8]:
mats, _ = G₂_matrices_roots();
mats[1]

14×14 Matrix{Int64}:
 1   0   0   0  0  0  -1  0  0  0  0  0  -2  1
 0   1   0   0  0  0   0  0  0  0  0  0   0  0
 0  -1   1   0  0  0   0  0  0  0  0  0   0  0
 0   1  -2   1  0  0   0  0  0  0  0  0   0  0
 0  -1   3  -3  1  0   0  0  0  0  0  0   0  0
 0   0   0   0  0  1   0  0  0  0  0  0   0  0
 0   0   0   0  0  0   1  0  0  0  0  0   0  0
 0   0   0   0  0  0   0  1  3  3  1  0   0  0
 0   0   0   0  0  0   0  0  1  2  1  0   0  0
 0   0   0   0  0  0   0  0  0  1  1  0   0  0
 0   0   0   0  0  0   0  0  0  0  1  0   0  0
 0   0   0   0  0  0   0  0  0  0  0  1   0  0
 0   0   0   0  0  0   1  0  0  0  0  0   1  0
 0   0   0   0  0  0   0  0  0  0  0  0   0  1

In [9]:
d = size(first(mats), 1)

14

In [10]:
G = MatrixGroups.MatrixGroup{d}(mats)

subgroup of 14×14 invertible matrices with 12 generators

## Weyl group of $G_2$

Finally we define a finite group of automorphisms of $G_2$ which act by permutations on the symmetric generating set. While the classical group is the group of reflections of the root system, our group, also generated by two reflections, is a $\mathbb{Z}/2\mathbb{Z}^n$-extension of the classical Weyl group.

In [11]:
S = let S = Groups.gens(G)
    union!(S, inv.(S)) # symmetric generating set
end

24-element Vector{FPGroupElement{Groups.MatrixGroups.MatrixGroup{14, Int64, DataType, Groups.MatrixGroups.MatrixElt{14, Int64, 196}}, KnuthBendix.Words.Word{UInt8}}}:
 m₁
 m₂
 m₃
 m₄
 m₅
 m₆
 m₇
 m₈
 m₉
 m₁₀
 m₁₁
 m₁₂
 m₁⁻¹
 m₂⁻¹
 m₃⁻¹
 m₄⁻¹
 m₅⁻¹
 m₆⁻¹
 m₇⁻¹
 m₈⁻¹
 m₉⁻¹
 m₁₀⁻¹
 m₁₁⁻¹
 m₁₂⁻¹

In [12]:
σ = let S = S, a = S[1], b = S[7]
    w = a * inv(b) * a
    # w is an element of G₂ which acts on S by conjugation:
    images = [findfirst(==(w^-1 * s * w), S) for s in S]
    Perm(images)
end

(1,19)(2,5,14,17)(3,16,15,4)(7,13)(8,11,20,23)(9,22,21,10)

In [13]:
τ = let S = S, a = S[2], b = S[8]
    w = a * inv(b) * a
    # w is an element of G₂ which acts on S by conjugation:
    images = [findfirst(==(w^-1 * s * w), S) for s in S]
    Perm(images)
end

(1,15,13,3)(2,20)(5,6,17,18)(7,21,19,9)(8,14)(11,12,23,24)

In [14]:
Weyl = PermGroup(σ, τ)

Permutation group on 2 generators generated by
 (1,19)(2,5,14,17)(3,16,15,4)(7,13)(8,11,20,23)(9,22,21,10)
 (1,15,13,3)(2,20)(5,6,17,18)(7,21,19,9)(8,14)(11,12,23,24)

In [15]:
Groups.order(Weyl)

48

(as compared to the classical Weyl group of order $12$ isomorphic to $D_6$).

# Sum of squares proof of property (T) for $\operatorname{G}_{2}(\mathbb{Z})$

We wish to prove
> **Theorem 3.17** Let $G$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{G}_{\texttt{2}}$ and let $S$ be the set of its Steinberg generators. The pair $(G, S)$ has property (T) with a witness of type $(\lambda, R) = (0.96768, 2)$.

We will show this by exhibiting $\xi_i\in \mathbb{R}G$, supported inside $\operatorname{Ball}(S, 2)$ such that

$$
\Delta^2 - \lambda \Delta - \sum_i \xi_i^* \xi_i = r,
$$

with $\|r\|_1$ much smaller (a few orders of magnitue) than $\lambda$.

In [16]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

[ Info: generating wl-metric ball of radius 4


  0.990229 seconds (524.37 k allocations: 35.080 MiB, 1.71% gc time)


[ Info: sizes = [25, 457, 7381, 110797]
[ Info: computing the *-algebra structure for G


  0.238904 seconds (926 allocations: 4.105 MiB)
  1.693567 seconds (1.52 M allocations: 108.028 MiB, 1.00% gc time, 27.36% compilation time)


In [17]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

24·(id) -1·m₁ -1·m₂ -1·m₃ -1·m₄ -1·m₅ -1·m₆ -1·m₇ -1·m₈ -1·m₉ -1·m₁₀ -1·m₁₁ -1·m₁₂ -1·m₁⁻¹ -1·m₂⁻¹ -1·m₃⁻¹ -1·m₄⁻¹ -1·m₅⁻¹ -1·m₆⁻¹ -1·m₇⁻¹ -1·m₈⁻¹ -1·m₉⁻¹ -1·m₁₀⁻¹ -1·m₁₁⁻¹ -1·m₁₂⁻¹

## Optimization problem

### Symmetry reduction

In [18]:
import PropertyT.SA as StarAlgebras
import PropertyT.SW as SymbolicWedderburn
using PropertyT.PG # PermutationGroups

In [19]:
wd = let Σ = Weyl, RG = RG
    act = PropertyT.AlphabetPermutation{eltype(Σ),Int64}(
        Dict(g => g for g in Σ),
    )

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Σ,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
        semisimple = false,
    )
end
@info wd

  5.345547 seconds (10.34 M allocations: 729.592 MiB, 2.26% gc time, 265.74% compilation time: 6% of which was recompilation)


┌ Info: Wedderburn Decomposition into 2393 orbits and 10 simple summands of sizes
└ [18, 13, 12, 8, 25, 25, 29, 25, 26, 22]


In [20]:
@time model, varP = PropertyT.sos_problem_primal(Δ^2, Δ, wd; augmented = true);
model

  5.269973 seconds (8.60 M allocations: 1.226 GiB, 2.64% gc time, 112.91% compilation time: <1% of which was recompilation)


A JuMP Model
Maximization problem with:
Variables: 2391
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2392 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 10 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

## Numerical approximation of the solution

In [21]:
include(joinpath(dirname(pathof(PropertyT)), "..", "test", "optimizers.jl"))

cosmo_optimizer (generic function with 1 method)

In [22]:
with_optimizer = scs_optimizer(;
    eps = 1e-9,
    max_iters = 20_000,
    accel = 50,
    alpha = 1.95,
);

In [23]:
warm = nothing

In [24]:
status, warm = PropertyT.solve(
    model, 
    with_optimizer, 
    warm,
);
@info "Optimization has finished with" status

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 2391, constraints m: 4782
cones: 	  z: primal zero / dual free vars: 2392
	  s: psd vars: 2390, ssize: 10
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.95, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 50, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 59894, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 4.82e+01  1.00e+00  2.81e+02 -2.29e+02  1.00e-01  2.80e-02 
   250| 2.05e-02  7.97e-04  2.88e-01 -2

┌ Info: Optimization has finished with
└   status = OPTIMAL::TerminationStatusCode = 1


## Reconstructing the solution and certification 

In [25]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

[ Info: reconstructing the solution


 11.467940 seconds (22.01 M allocations: 1.329 GiB, 2.81% gc time, 101.16% compilation time)


457×457 Matrix{Float64}:
 0.0   0.0           0.0           0.0          …   0.0          0.0
 0.0   2.13548       0.0800336     0.151            0.0277127   -0.00333869
 0.0   0.0800336     4.072        -0.000880108     -0.00150363  -0.0146724
 0.0   0.151        -0.000880108   2.13548         -0.00338313  -0.0194072
 0.0   0.0637425     0.123988      0.0637425       -0.00338313  -0.0194072
 0.0  -0.000880108   0.0234846     0.123988     …  -0.00150363  -0.0146724
 0.0   0.123988     -0.0135709    -0.000880108      0.00177502  -0.0212883
 0.0   0.0748156    -0.000880108   0.0637425        0.0208018    0.0236174
 0.0  -0.000880108  -0.0998939     0.0800336        0.00911572   0.0194903
 0.0   0.0637425     0.0800336     0.0748156       -0.00866713  -0.0385493
 0.0   0.151         0.0040989     0.151        …  -0.00866713  -0.0385493
 0.0   0.0800336    -0.0135709     0.0040989        0.00911572   0.0194903
 0.0   0.0040989     0.0234846     0.0800336        0.024449     0.00126253
 ⋮  

In [26]:
@info "certifying the solution"
@time certified, λ = PropertyT.certify_solution(
    Δ^2,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)
if certified && λ > 0
    Κ(λ, S) = sqrt(2λ/length(S))
    @info "Certified result: G₂ has property (T):" inf(λ) inf(Κ(λ, S))
else
    @info "Could NOT certify property (T) for G₂" certified λ
end

[ Info: certifying the solution
┌ Info: Checking in Float64 arithmetic with
└   λ = 0.9676852341440747


  0.002837 seconds (6 allocations: 2.439 MiB)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ 4.44723e-11
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 1.68247e-5
└  λ ≈ 0.9676179351445554
┌ Info: Checking in Interval{Float64} arithmetic with
└   λ_int = [0.967685, 0.967686]


  0.271747 seconds (225.63 k allocations: 54.622 MiB, 88.58% compilation time)
  4.017271 seconds (6.25 M allocations: 459.867 MiB, 2.03% gc time, 98.93% compilation time)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ∈ [-1.13147e-08, 1.12118e-08]
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ∈ [1.68135e-05, 1.68362e-05]
└  λ ∈ [0.967617, 0.967618]
┌ Info: Certified result: G₂ has property (T):
│   inf(λ) = 0.9676178897084275
└   inf(Κ(λ, S)) = 0.2839627161131703


# $\texttt{G}_\texttt{2}$-graded $\operatorname{Adj}$

We wish to prove

> **Theorem 3.18** Let $G$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{G}_\texttt{2}$ and let $S$ be the set of its Steinberg generators. Let $V$ denote the ambient vector space of the root system. Then 
>
> $$\operatorname{Adj}_V −\lambda \Delta_V ⩾_R 0$$
>
>for $(\lambda, R) \in (1.56799, 3)$.


**BIG FAT WARNING**: Proving this theorem for $R = 3$ requires **lots of memory** (`>32GB`) and **even more patience**. This is due to the following facts.
* That there are more than $22\,000\,000$ elements in the ball of radius $6$ while the Weyl groups is fairly small, with just `48` elements. This results in Wedderburn Decomposition into `459849` orbits and `10` simple summands of sizes `[334, 332, 182, 167, 165, 153, 459, 447, 449, 439]` (fairly large PSD constraints).
 * Simply generating the data to formulate the optimization problem takes more than `2h` on a workstation computer.
 * Running `scs` solver on the problem for `100_000` iterations takes more than `24h`.

If you have the necessary technical requirements and enough grit you may change `HALFRADIUS` to `3` below. `HALFRADIUS = 2` will not allow  to obtain a positive result!

In [27]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

[ Info: generating wl-metric ball of radius 4


  0.996501 seconds (524.37 k allocations: 35.080 MiB)


[ Info: sizes = [25, 457, 7381, 110797]
[ Info: computing the *-algebra structure for G


  0.240097 seconds (926 allocations: 4.105 MiB)
  1.237285 seconds (526.10 k allocations: 39.251 MiB)


In [28]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

24·(id) -1·m₁ -1·m₂ -1·m₃ -1·m₄ -1·m₅ -1·m₆ -1·m₇ -1·m₈ -1·m₉ -1·m₁₀ -1·m₁₁ -1·m₁₂ -1·m₁⁻¹ -1·m₂⁻¹ -1·m₃⁻¹ -1·m₄⁻¹ -1·m₅⁻¹ -1·m₆⁻¹ -1·m₇⁻¹ -1·m₈⁻¹ -1·m₉⁻¹ -1·m₁₀⁻¹ -1·m₁₁⁻¹ -1·m₁₂⁻¹

## Defining $\texttt{G}_\texttt{2}$-grading

Through GAP, we obtain the set of roots of $G_2$ corresponding to `all_gens` (as above):
```GAP
gap> roots := ShallowCopy(PositiveRoots(g2rs));; Append(roots, NegativeRoots(g2rs));; roots;
[ [ 2, -1 ], [ -3, 2 ], [ -1, 1 ], [ 1, 0 ], [ 3, -1 ], [ 0, 1 ], [ -2, 1 ], [ 3, -2 ], [ 1, -1 ], [ -1, 0 ], [ -3, 1 ], [ 0, -1 ] ]
```

These roots are the ones from the Cartan matrix. To obtain the standard (more hexagonal) picture map them by `T` defined as follows:
```julia
cartan = [ 2 -3 ;
          -1  2 ]
rot(α) = [cos(α) -sin(α); sin(α) cos(α)]

c₁ = [√2, 0]
c₂ = rot(5π / 6) * [√2, 0] * √3 # (= 1/2[√6, 1])

T = hcat(c₁, c₂) * inv(cartan)
```
By plotting one against the others (or by blind calculation) one can see the following assignment:

```julia
G₂roots_gap = [
    [2, -1], # α = e₁ - e₂
    [-3, 2], # A = -α + β = -e₁ + 2e₂ - e₃
    [-1, 1], # β = e₂ - e₃
    [1, 0], # α + β = e₁ - e₃
    [3, -1], # B = 2α + β = 2e₁ - e₂ - e₃
    [0, 1], # A + B = α + 2β = e₁ + e₂ - 2e₃
    [-2, 1], # -α
    [3, -2], # -A
    [1, -1], # -β
    [-1, 0], # -α - β
    [-3, 1], # -B
    [0, -1], # -A - B
]
```

One can see that $\langle \alpha, \beta \rangle_\mathbb{Z} = \texttt{A}_\texttt{2}$ and 
$\langle A, B \rangle_\mathbb{Z} = \frac{\sqrt{3}}{\sqrt{2}}\texttt{A}_\texttt{2}$.

The roots corresponding to our generators are therefore of the following form.

In [29]:
using PropertyT.Roots
e₁ = PropertyT.Roots.𝕖(3, 1)
e₂ = PropertyT.Roots.𝕖(3, 2)
e₃ = PropertyT.Roots.𝕖(3, 3)

α = e₁ - e₂
β = e₂ - e₃
A = -α + β
B = α + (α + β)

roots = [α, A, β, α + β, B, A + B, -α, -A, -β, -α - β, -B, -A - B]

12-element Vector{Root{3, Int64}}:
 Root [1, -1, 0]
 Root [-1, 2, -1]
 Root [0, 1, -1]
 Root [1, 0, -1]
 Root [2, -1, -1]
 Root [1, 1, -2]
 Root [-1, 1, 0]
 Root [1, -2, 1]
 Root [0, -1, 1]
 Root [-1, 0, 1]
 Root [-2, 1, 1]
 Root [-1, -1, 2]

In [30]:
G₂grading = let A = alphabet(G), grading = Dict{eltype(A), eltype(roots)}()  
    for (root, g) in zip(roots, gens(G))
        letter = first(word(g))
        # assigning root to both g and g⁻¹:
        grading[A[letter]] = root
        grading[A[inv(letter, A)]] = root
    end
    grading 
end

Dict{Groups.MatrixGroups.MatrixElt{14, Int64, 196}, Root{3, Int64}} with 24 entries:
  m₁    => Root [1, -1, 0]
  m₃    => Root [0, 1, -1]
  m₃⁻¹  => Root [0, 1, -1]
  m₈⁻¹  => Root [1, -2, 1]
  m₄⁻¹  => Root [1, 0, -1]
  m₇⁻¹  => Root [-1, 1, 0]
  m₈    => Root [1, -2, 1]
  m₉⁻¹  => Root [0, -1, 1]
  m₆⁻¹  => Root [1, 1, -2]
  m₁₀⁻¹ => Root [-1, 0, 1]
  m₁⁻¹  => Root [1, -1, 0]
  m₁₁   => Root [-2, 1, 1]
  m₁₁⁻¹ => Root [-2, 1, 1]
  m₉    => Root [0, -1, 1]
  m₅    => Root [2, -1, -1]
  m₁₀   => Root [-1, 0, 1]
  m₁₂   => Root [-1, -1, 2]
  m₇    => Root [-1, 1, 0]
  m₂⁻¹  => Root [-1, 2, -1]
  m₅⁻¹  => Root [2, -1, -1]
  m₄    => Root [1, 0, -1]
  m₂    => Root [-1, 2, -1]
  m₆    => Root [1, 1, -2]
  m₁₂⁻¹ => Root [-1, -1, 2]

In [31]:
function PropertyT.grading(g::MatrixGroups.MatrixElt, grading = G₂grading)
    return grading[g]
end

In [32]:
g = gens(G,1)

m₁ ∈ H ⩽ GL{14,Int64}
 1   0   0   0  0  0  -1  0  0  0  0  0  -2  1
 0   1   0   0  0  0   0  0  0  0  0  0   0  0
 0  -1   1   0  0  0   0  0  0  0  0  0   0  0
 0   1  -2   1  0  0   0  0  0  0  0  0   0  0
 0  -1   3  -3  1  0   0  0  0  0  0  0   0  0
 0   0   0   0  0  1   0  0  0  0  0  0   0  0
 0   0   0   0  0  0   1  0  0  0  0  0   0  0
 0   0   0   0  0  0   0  1  3  3  1  0   0  0
 0   0   0   0  0  0   0  0  1  2  1  0   0  0
 0   0   0   0  0  0   0  0  0  1  1  0   0  0
 0   0   0   0  0  0   0  0  0  0  1  0   0  0
 0   0   0   0  0  0   0  0  0  0  0  1   0  0
 0   0   0   0  0  0   1  0  0  0  0  0   1  0
 0   0   0   0  0  0   0  0  0  0  0  0   0  1

In [33]:
PropertyT.grading(g)

Root in ℝ^3 of length √2
[1, -1, 0]

In [34]:
PropertyT.grading(inv(g))

Root in ℝ^3 of length √2
[1, -1, 0]

In [35]:
g = gens(G, 2)

m₂ ∈ H ⩽ GL{14,Int64}
 1  0  0  0   0  0  0   0   0  0  0  0  0   0
 0  1  0  0   0  0  0  -1   0  0  0  0  3  -2
 1  0  1  0   0  0  0   0   0  0  0  0  0   0
 0  0  0  1   0  0  0   0   0  0  0  0  0   0
 0  0  0  0   1  0  0   0   0  0  0  0  0   0
 0  0  0  0  -1  1  0   0   0  0  0  0  0   0
 0  0  0  0   0  0  1   0  -1  0  0  0  0   0
 0  0  0  0   0  0  0   1   0  0  0  0  0   0
 0  0  0  0   0  0  0   0   1  0  0  0  0   0
 0  0  0  0   0  0  0   0   0  1  0  0  0   0
 0  0  0  0   0  0  0   0   0  0  1  1  0   0
 0  0  0  0   0  0  0   0   0  0  0  1  0   0
 0  0  0  0   0  0  0   0   0  0  0  0  1   0
 0  0  0  0   0  0  0   1   0  0  0  0  0   1

In [36]:
PropertyT.grading(g)

Root in ℝ^3 of length √6
[-1, 2, -1]

In [37]:
Δs = PropertyT.laplacians(
    RG,
    S,
    x -> (gx = PropertyT.grading(x); Set([gx, -gx])),
);

Here `Δs` is just a map from lines in the root system $\Omega = \texttt{G}_{\texttt{2}}$ to the corresponding Laplacians, e.g. below we can see that to the line through `α = [1, -1, 0]` and `-α` (and the origin) we assign
$$ \Delta_{Lα} = 4 - m_{1} - m_{7} - m_{1}^{-1} - m_{7}^{-1}.$$ 

In [38]:
using PropertyT.Roots
let α = Root([1,-1, 0])
    Lα = Set([α, -α])
    Δs[Lα]
end

4·(id) -1·m₁ -1·m₇ -1·m₁⁻¹ -1·m₇⁻¹

In [39]:
using PropertyT.Roots
let α = Root([-1,2, -1])
    Lα = Set([α, -α])
    Δs[Lα]
end

4·(id) -1·m₂ -1·m₈ -1·m₂⁻¹ -1·m₈⁻¹

Following the definition of $\operatorname{Adj}$ we define
$$ \operatorname{Adj}_{\texttt{G}_\texttt{2}} = 
\prod_{
    \langle L\alpha, L\beta \rangle \cap \Omega \cong \texttt{G}_{\texttt{2}}
} \Delta_{L\alpha} \Delta_{L\beta} $$

In [40]:
AdjG₂ = PropertyT.Adj(Δs, :G₂)

480·(id) -40·m₁ -40·m₂ -40·m₃ -40·m₄ -40·m₅ -40·m₆ -40·m₇ -40·m₈ -40·m₉ -40·m₁₀ -40·m₁₁ -40·m₁₂ -40·m₁⁻¹ -40·m₂⁻¹ -40·m₃⁻¹ -40·m₄⁻¹ -40·m₅⁻¹ -40·m₆⁻¹ -40·m₇⁻¹ -40·m₈⁻¹ -40·m₉⁻¹ -40·m₁₀⁻¹ -40·m₁₁⁻¹ -40·m₁₂⁻¹ +1·m₁*m₂ +1·m₁*m₃ +1·m₁*m₄ +2·m₁*m₅ +2·m₁*m₆ +2·m₁*m₈ +1·m₁*m₉ +1·m₁*m₁₀ +1·m₁*m₁₁ +2·m₁*m₁₂ +1·m₁*m₂⁻¹ +1·m₁*m₃⁻¹ +1·m₁*m₄⁻¹ +2·m₁*m₅⁻¹ +2·m₁*m₆⁻¹ +2·m₁*m₈⁻¹ +1·m₁*m₉⁻¹ +1·m₁*m₁₀⁻¹ +1·m₁*m₁₁⁻¹ +2·m₁*m₁₂⁻¹ +1·m₂*m₁ +2·m₂*m₃ +2·m₂*m₄ +1·m₂*m₅ +2·m₂*m₆ +2·m₂*m₇ +1·m₂*m₉ +2·m₂*m₁₀ +2·m₂*m₁₁ +1·m₂*m₁₂ +1·m₂*m₁⁻¹ +2·m₂*m₃⁻¹ +2·m₂*m₄⁻¹ +1·m₂*m₅⁻¹ +2·m₂*m₆⁻¹ +2·m₂*m₇⁻¹ +1·m₂*m₉⁻¹ +2·m₂*m₁₀⁻¹ +2·m₂*m₁₁⁻¹ +1·m₂*m₁₂⁻¹ +1·m₃*m₁ +1·m₃*m₄ +2·m₃*m₅ +2·m₃*m₆ +1·m₃*m₇ +1·m₃*m₈ +1·m₃*m₁₀ +2·m₃*m₁₁ +1·m₃*m₁₂ +1·m₃*m₁⁻¹ +2·m₃*m₂⁻¹ +1·m₃*m₄⁻¹ +2·m₃*m₅⁻¹ +2·m₃*m₆⁻¹ +1·m₃*m₇⁻¹ +1·m₃*m₈⁻¹ +1·m₃*m₁₀⁻¹ +2·m₃*m₁₁⁻¹ +1·m₃*m₁₂⁻¹ +1·m₄*m₁ +1·m₄*m₃ +2·m₄*m₅ +2·m₄*m₆ +1·m₄*m₇ +2·m₄*m₈ +1·m₄*m₉ +1·m₄*m₁₁ +1·m₄*m₁₂ +1·m₄*m₁⁻¹ +2·m₄*m₂⁻¹ +1·m₄*m₃⁻¹ +2·m₄*m₅⁻¹ +2·m₄*m₆⁻¹ +1·m₄*m₇⁻¹ +2·m₄*m₈⁻¹ +1·m₄*m₉⁻¹ +1·m₄*m₁₁⁻¹ +

It is not hard to see that for $\Omega = \texttt{G}_{\texttt{2}}$ 
 * we are simply looking at products of all $\Delta_{L\alpha}$ and $\Delta_{L\beta}$ where $L\alpha \neq L\beta$, and
 * that the new definition is an analouge to the definition of $\operatorname{Adj}$ from [On property (T) for $\operatorname{Aut}(F_n)$ and $\operatorname{SL}_n(\mathbb{Z})$](https://arxiv.org/abs/1812.03456).

In [41]:
AdjG₂ == Δ^2 - sum(Δs[Lα]^2 for Lα in keys(Δs))

true

## Optimization problem
### Symmetry reduction


In [42]:
import PropertyT.SA as StarAlgebras
import PropertyT.SW as SymbolicWedderburn
using PropertyT.PG # PermutationGroups

In [43]:
wd = let Σ = Weyl, RG = RG
    act = PropertyT.AlphabetPermutation{eltype(Σ),Int64}(
        Dict(g => PropertyT.PG.AP.perm(g) for g in Σ),
    )

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Σ,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
        semisimple = false,
    )
end
@info wd

  0.175309 seconds (658.77 k allocations: 79.936 MiB, 167.12% compilation time: 21% of which was recompilation)


┌ Info: Wedderburn Decomposition into 2393 orbits and 10 simple summands of sizes
└ [18, 13, 12, 8, 25, 25, 29, 25, 26, 22]


In [44]:
@time model, varP = PropertyT.sos_problem_primal(AdjG₂, Δ, wd; augmented = true);
model

  2.370213 seconds (2.17 M allocations: 816.659 MiB, 3.16% gc time)


A JuMP Model
Maximization problem with:
Variables: 2391
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2392 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 10 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

## Solving the problem numerically
We will use `scs` [Splitting Conic Solver](https://github.com/cvxgrp/scs) so solve this problem.

In [45]:
include(joinpath(@__DIR__, "..", "src", "optimizers.jl"));

In [46]:
with_optimizer = scs_optimizer(;
    eps = 1e-9,
    max_iters = 100_000,
    accel = 50,
    alpha = 1.95,
);

In [47]:
warm = nothing

> **Note** If you survived until now with `HALFRADIUS = 3`...
> * To obtain just **any positive lower bound** it is advisable to (artificially) bound the objective from above, e.g. by adding
    ```julia
    JuMP.@constraint(model, upper_bound, model[:λ] ≤ 1.0)
    ```
    before solving the model (to bring the solve time to below 1h).
> * If you do not bound the objective you will need to re-run the cell below several (a dozen? times to obtain `status = OPTIMAL::TerminationStatusCode = 1`. To succesfully certify **a lower bound** that might not be necessary. However this will be necessary to obtain **the bound advertised** in the paper.


In [48]:
if HALFRADIUS == 3
    JuMP.@constraint(model, upper_bound, model[:λ] ≤ 1.0)
end

In [49]:
status, warm = PropertyT.solve(
        model,
        with_optimizer,
        warm,
    );
# note: since we're using scs there will be no printout until the optimization has finished 
# please bear with us...
@info "Optimization has finished with" status

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 2391, constraints m: 4782
cones: 	  z: primal zero / dual free vars: 2392
	  s: psd vars: 2390, ssize: 10
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.95, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 50, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-mkl-pardiso
	  nnz(A): 59894, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 4.02e+01  1.00e+00  2.80e+02 -2.13e+02  1.00e-01  3.77e-02 
   250| 2.56e-02  5.53e-04  2.00e-01

┌ Info: Optimization has finished with
└   status = OPTIMAL::TerminationStatusCode = 1


### Reconstructing and certifying the solution

In [50]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

  0.052168 seconds (125.78 k allocations: 24.664 MiB, 85.19% compilation time)


[ Info: reconstructing the solution


457×457 Matrix{Float64}:
 0.0   0.0          0.0           0.0          …   0.0           0.0
 0.0   2.14167      0.0714181     0.132649        -0.00701363    0.00177425
 0.0   0.0714181    3.69845       0.00761566       0.00638768   -0.0106907
 0.0   0.132649     0.00761566    2.14167          0.00379035   -0.0150442
 0.0   0.0497086    0.118509      0.0497086        0.00379035   -0.0150442
 0.0   0.00761566   0.0263223     0.118509     …   0.00638768   -0.0106907
 0.0   0.118509     0.0896173     0.00761566       0.000279809   0.00042273
 0.0   0.0539916    0.00761566    0.0497086        0.00271279   -0.0116439
 0.0   0.00761566  -0.0280579     0.0714181        0.0174868     0.016446
 0.0   0.0497086    0.0714181     0.0539916        0.0108266    -0.0382329
 0.0   0.132649    -0.0189233     0.132649     …   0.0108266    -0.0382329
 0.0   0.0714181    0.0896173    -0.0189233        0.0174868     0.016446
 0.0  -0.0189233    0.0263223     0.0714181        5.12445e-5    0.00776396
 ⋮   

In [51]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    AdjG₂,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Adj_C₂ is positive" PropertyT.IntervalArithmetic.inf(λ)
else
    @info "Could NOT certify the positivity of Adj_C₂" certified λ
end

  0.002188 seconds (6 allocations: 2.439 MiB)


[ Info: certifying the solution
┌ Info: Checking in Float64 arithmetic with
└   λ = -0.881658358205522
┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ -5.06501e-11
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 1.35469e-5
└  λ ≈ -0.8817125457899869
┌ Info: Could NOT certify the positivity of Adj_C₂
│   certified = false
└   λ = -0.8817125457899869


If you solved the problem with `HALFRADIUS = 2`, then you might notice that the optimal $\lambda$ that we obtained is `-0.881...` i.e. **negative**. This means that not only $\operatorname{Adj}_{\texttt{G}_{\texttt{2}}}$ is not positive (is not a sum of squares), but one has to **add** almost a whole $\Delta$ to obtain a positive element. In other words

$$
\operatorname{Adj}_{\texttt{G}_{\texttt{2}}} + 0.881...\Delta = \sum_i \xi_i^* \xi_i, \quad \operatorname{supp}{\xi_i} \subseteq \operatorname{Ball}(S, 2).
$$

Passing to `HALFRADIUS = 3` allows us to obtain a positive result i.e.
$$
\operatorname{Adj}_{\texttt{G}_{\texttt{2}}} - 1.568...\Delta = \sum_i \xi_i^* \xi_i, \quad \operatorname{supp}{\xi_i} \subseteq \operatorname{Ball}(S, 3).
$$

In [52]:
using Dates
Dates.now()

2024-07-08T14:54:43.400

In [53]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 7840U w/ Radeon  780M Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver3)
Threads: 8 default, 0 interactive, 4 GC (on 16 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
